Definition of `image_taken_date` function:

In [206]:
import logging
from pathlib import Path

def image_taken_date(file):
    """Attempts to retrieve original date and time from a photo
    input: string or result of pathlib
    output: datetime object """
    from datetime import datetime

    # First try to return the file date and time using the exifread library
    try:
        import exifread
        f = open(file, 'rb')
        exif_data = exifread.process_file(f, stop_tag='EXIF DateTimeOriginal')
        exif_datetime = exif_data['EXIF DateTimeOriginal']
        return datetime.strptime(exif_datetime.values, '%Y:%m:%d %H:%M:%S')
    except: 
        pass
    
    # If that did not work, try pillow
    try:
        import PIL.Image
        img = PIL.Image.open(str(file))
        exif_data = img.getexif()
        exif_datetime = exif_data['EXIF DateTimeOriginal']
        datetime_str = f'{exif_datetime}'
        return datetime.strptime(datetime_str, '%Y:%m:%d %H:%M:%S')
    except:
        pass

    # If that did also not work, try os
    result = datetime.fromtimestamp(file.stat().st_atime)
    return result


Definition of ```image_size``` function:

In [207]:
def image_size(file):
    # First try PIL since it is most likely installed on the computer
    try:
        import PIL.Image
        img = PIL.Image.open(file)
        return img.size
    except:
        pass
    
    # The use exifread since PIL may not work on ARW files
    try:
        import exifread
        f = open(file, 'rb')
        exifdata = exifread.process_file(f)
        width = exifdata['EXIF ExifImageWidth'].values[0]
        height = exifdata['EXIF ExifImageLength'].values[0]
        return (width, height)
    except:
        return (0,0)

Retrieve a list of files from disk:

In [208]:
from pathlib import Path, PurePath
path = Path( Path.home(), "Downloads", "Source")
files = [file for file in path.iterdir()]
print('\n'.join([f"[{index:03d}] {file.name}" for index, file in enumerate(files)]))

[000] 20190426-0048-2.CR2
[001] 20190426-0048-3.CR2
[002] DSC00768.JPG
[003] DSC00769.ARW
[004] DSC00769.JPG
[005] DSC01769.JPG
[006] DSC01770.JPG


Get EXIF data from PIL:

In [209]:
import PIL.Image

def exif(file):
    try:
        img = PIL.Image.open(file)
        return img.getexif()
    except:
        pass
    return {306: 'some kind of error'}

print('\n'.join([f"[{index:03d}] {file.name:24s} {exif(str(file))[306]} " for index, file in enumerate(files)]))


[000] 20190426-0048-2.CR2      2019:04:26 13:52:31 
[001] 20190426-0048-3.CR2      2019:04:26 13:52:57 
[002] DSC00768.JPG             2022:04:30 09:27:09 
[003] DSC00769.ARW             some kind of error 
[004] DSC00769.JPG             2022:04:30 09:27:14 
[005] DSC01769.JPG             2022:08:14 16:00:38 
[006] DSC01770.JPG             2022:08:14 16:02:39 


Overview of keys in PIL exif dictionary:

In [210]:
index = 2
pil_exif_dict = exif(str(files[index]))
list(pil_exif_dict.keys())

[50341, 296, 34665, 270, 271, 272, 305, 274, 306, 531, 282, 283]

In [211]:
print('\n'.join([f"{key:6d} {pil_exif_dict[key]}" for key in pil_exif_dict]))

 50341 b'PrintIM\x000300\x00\x00\x03\x00\x02\x00\x01\x00\x00\x00\x03\x00"\x00\x00\x00\x01\x01\x00\x00\x00\x00\t\x11\x00\x00\x10\'\x00\x00\x0b\x0f\x00\x00\x10\'\x00\x00\x97\x05\x00\x00\x10\'\x00\x00\xb0\x08\x00\x00\x10\'\x00\x00\x01\x1c\x00\x00\x10\'\x00\x00^\x02\x00\x00\x10\'\x00\x00\x8b\x00\x00\x00\x10\'\x00\x00\xcb\x03\x00\x00\x10\'\x00\x00\xe5\x1b\x00\x00\x10\'\x00\x00'
   296 2
 34665 364
   270                                
   271 SONY
   272 ILCE-6400
   305 ILCE-6400 v1.00
   274 1
   306 2022:04:30 09:27:09
   531 2
   282 350.0
   283 350.0


Determine size of image using PIL

In [212]:
import PIL.Image

def pil_size(file):
    try:
        img = PIL.Image.open(file)
        return img.size
    except:
        pass
    return (0,0)

print('\n'.join([f"{file.name:24s} {pil_size(str(file))}" for file in files]))

20190426-0048-2.CR2      (5184, 3456)
20190426-0048-3.CR2      (5184, 3456)
DSC00768.JPG             (6000, 3376)
DSC00769.ARW             (0, 0)
DSC00769.JPG             (6000, 3376)
DSC01769.JPG             (6000, 3376)
DSC01770.JPG             (6000, 3376)


In [213]:
mdate = lambda file : datetime.fromtimestamp( file.stat().st_atime ).strftime('%Y-%m-%d')
cdate = lambda file : datetime.fromtimestamp( file.stat().st_ctime ).strftime('%Y-%m-%d')
adate = lambda file : datetime.fromtimestamp( file.stat().st_atime ).strftime('%Y-%m-%d')

filedates = [f'[{index:03d}] {file.name:24s} modified:{mdate(file)}  created:{cdate(file)}  accessed:{adate(file)}' for index, file in enumerate(files)]
             
print('\n'.join(filedates))


[000] 20190426-0048-2.CR2      modified:2023-10-03  created:2023-10-02  accessed:2023-10-03
[001] 20190426-0048-3.CR2      modified:2023-10-03  created:2023-10-02  accessed:2023-10-03
[002] DSC00768.JPG             modified:2023-10-03  created:2023-10-02  accessed:2023-10-03
[003] DSC00769.ARW             modified:2023-10-03  created:2023-10-02  accessed:2023-10-03
[004] DSC00769.JPG             modified:2023-10-03  created:2023-10-02  accessed:2023-10-03
[005] DSC01769.JPG             modified:2023-10-03  created:2023-10-02  accessed:2023-10-03
[006] DSC01770.JPG             modified:2023-10-03  created:2023-10-02  accessed:2023-10-03


Explore exifread library:

In [214]:
index = 3
file = str(files[index])
f = open(file, 'rb')
#exif_data = exifread.process_file(f, stop_tag='EXIF DateTimeOriginal')
exifdata = exifread.process_file(f)
print('\n'.join([f"{str(key)}" for key in exifdata.keys()]))

Image SubfileType
Image Compression
Image ImageDescription
Image Make
Image Model
Image Orientation
Image XResolution
Image YResolution
Image ResolutionUnit
Image Software
Image DateTime
Image SubIFDs
Image JPEGInterchangeFormat
Image JPEGInterchangeFormatLength
Image YCbCrPositioning
Image ApplicationNotes
Image ExifOffset
Image PrintIM
Image Tag 0xC634
Thumbnail SubfileType
Thumbnail Compression
Thumbnail ImageDescription
Thumbnail Make
Thumbnail Model
Thumbnail Orientation
Thumbnail XResolution
Thumbnail YResolution
Thumbnail ResolutionUnit
Thumbnail Software
Thumbnail DateTime
Thumbnail JPEGInterchangeFormat
Thumbnail JPEGInterchangeFormatLength
Thumbnail YCbCrPositioning
EXIF ExposureTime
EXIF FNumber
EXIF ExposureProgram
EXIF ISOSpeedRatings
EXIF SensitivityType
EXIF RecommendedExposureIndex
EXIF ExifVersion
EXIF DateTimeOriginal
EXIF DateTimeDigitized
EXIF OffsetTime
EXIF OffsetTimeOriginal
EXIF OffsetTimeDigitized
EXIF ComponentsConfiguration
EXIF CompressedBitsPerPixel
EXIF Br

Explore getting image date from exifread library

In [215]:
tags = ['Image DateTime', 'EXIF DateTimeOriginal', 'EXIF DateTimeDigitized', 
        'EXIF ExifImageWidth', 'EXIF ExifImageLength']
print('\n'.join([f'{tag:32s} {exifdata[tag]}' for tag in tags] ))

Image DateTime                   2022:04:30 09:27:14
EXIF DateTimeOriginal            2022:04:30 09:27:14
EXIF DateTimeDigitized           2022:04:30 09:27:14
EXIF ExifImageWidth              6000
EXIF ExifImageLength             3376


In [216]:
type(exifdata['EXIF DateTimeOriginal'])

exifread.classes.IfdTag

Explore getting image size from exifread library

In [217]:
index = 3
file = files[index]
f = open(file, 'rb')
exifdata = exifread.process_file(f)
width = exifdata['EXIF ExifImageWidth']
height = exifdata['EXIF ExifImageLength']
#print(width,height)
#dir(exifdata['EXIF ExifImageWidth'])
exifdata['EXIF ExifImageWidth'].values[0]

6000

In [218]:
import exifread
def exifread_size(file):
    f = open(file, 'rb')
    exifdata = exifread.process_file(f)
    width = exifdata['EXIF ExifImageWidth'].values[0]
    height = exifdata['EXIF ExifImageLength'].values[0]
    return (width, height)

print('\n'.join([f"{file.name:24s} {exifread_size(file)} {pil_size(str(file))}" for file in files]))

20190426-0048-2.CR2      (5184, 3456) (5184, 3456)
20190426-0048-3.CR2      (5184, 3456) (5184, 3456)
DSC00768.JPG             (6000, 3376) (6000, 3376)
DSC00769.ARW             (6000, 3376) (0, 0)
DSC00769.JPG             (6000, 3376) (6000, 3376)
DSC01769.JPG             (6000, 3376) (6000, 3376)
DSC01770.JPG             (6000, 3376) (6000, 3376)


Testing the generic ```image_size``` function to retrieve the size of an image:

In [219]:
print('\n'.join([f"{file.name:24s} {image_size(file)}" for file in files]))

20190426-0048-2.CR2      (5184, 3456)
20190426-0048-3.CR2      (5184, 3456)
DSC00768.JPG             (6000, 3376)
DSC00769.ARW             (6000, 3376)
DSC00769.JPG             (6000, 3376)
DSC01769.JPG             (6000, 3376)
DSC01770.JPG             (6000, 3376)


In [220]:
string_to_date = lambda datetime_str : datetime.strptime(datetime_str, '%Y:%m:%d %H:%M:%S')
date_to_string = lambda d : datetime.fromtimestamp( d ).strftime('%Y-%m-%d')
s = f"{exifdata['EXIF DateTimeOriginal']}"
date = string_to_date(s)
print(date.strftime('%Y-%m-%d'))


2022-04-30


In [221]:
print('\n'.join([f"{str(key):32s} {str(exifdata[key])}" for key in exifdata.keys()]))


Image SubfileType                Reduced-resolution image
Image Compression                JPEG (old-style)
Image ImageDescription                                          
Image Make                       SONY
Image Model                      ILCE-6400
Image Orientation                Horizontal (normal)
Image XResolution                350
Image YResolution                350
Image ResolutionUnit             Pixels/Inch
Image Software                   ILCE-6400 v1.00
Image DateTime                   2022:04:30 09:27:14
Image SubIFDs                    132288
Image JPEGInterchangeFormat      135330
Image JPEGInterchangeFormatLength 545838
Image YCbCrPositioning           Co-sited
Image ApplicationNotes           []
Image ExifOffset                 4544
Image PrintIM                    [80, 114, 105, 110, 116, 73, 77, 0, 48, 51, 48, 48, 0, 0, 3, 0, 2, 0, 1, 0, ... ]
Image Tag 0xC634                 [254, 204, 0, 0]
Thumbnail SubfileType            Reduced-resolution image
Thumbnail Co

In [222]:
from pathlib import Path, PurePath
from datetime import datetime

def pil_date(file):
    import PIL.Image
    img = PIL.Image.open(str(file))
    exif_data = img.getexif()
    datetime_str = exif_data[306]        
    return datetime.strptime(datetime_str, '%Y:%m:%d %H:%M:%S')

datetime_to_string = lambda d : d.strftime('%Y-%m-%d %H:%M:%S')

path = Path( Path.home(), "Downloads", "Source")
files = [file for file in path.iterdir()]

for file in files:
    print(f'{file.name:24s}')
    print(f'{datetime_to_string(image_taken_date(file)):>40s}')


20190426-0048-2.CR2     
                     2019-04-26 13:52:31
20190426-0048-3.CR2     
                     2019-04-26 13:52:57
DSC00768.JPG            
                     2022-04-30 09:27:09
DSC00769.ARW            
                     2022-04-30 09:27:14
DSC00769.JPG            
                     2022-04-30 09:27:14
DSC01769.JPG            
                     2022-08-14 16:00:38
DSC01770.JPG            
                     2022-08-14 16:02:39


In [223]:
import exifread
index = 4
f = open(files[index], 'rb')
exif_data = exifread.process_file(f)
exif_datetime = exif_data['EXIF DateTimeOriginal']
datetime_str = f'{exif_datetime}'
result = datetime.strptime(datetime_str, '%Y:%m:%d %H:%M:%S')
#dir(exif_datetime)
exif_datetime.values

'2022:04:30 09:27:14'

In [224]:
import PIL.Image
index = 4
img = PIL.Image.open(files[index])
exif_data = img.getexif()
exif_data[306]

'2022:04:30 09:27:14'

In [5]:
my_dict = { 'A': 1}

#my_dict.has_key('A')
#dir(my_dict)
'A' in my_dict.keys()

True